# Trabalho Final referente a disciplina de Data Science 
### Aluno: Alves, Carlos F.C.

## Objetivo

Este trabalho faz a analise exploratória dos resultados obtidos do trabalho de mestrado sobre caracterização estatística da transição da convecção rasa para profunda ao longo do experimento GoAmazon2014/15.


## Introdução
Em relação a trabalhos anteriores, o trabalho de mestrado, propôs um critério para identificação da convecção profunda contornando o limite experimental para detecção de vapor dágua e aproveitando a resolução do GOES-13 além da instrumentação disponível ao longo do período.

In [1]:
import pandas as pd
import numpy as np
import csv
import re
import datetime
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
plt.style.use('ggplot')
%matplotlib inline

## INPUT FILES

The previous work was accomplished using Matlab, so I did a matlab script to export:
- [arm_vars.csv]   : Variable names of Time Series
- [arm_values.csv] : ARM DOE Time Series that was resampled by 5min and set NAN values to ones that dont have a good quality check flag provided by ARM 
- [arm_time.csv]   : date_time of each TS values in Local Time Units
- [convective_dates.csv] : convective events selected by previous work methodology

Original Data Downloaded from: http://www.archive.arm.gov/discovery/#v/results/s/



## LOAD : [arm_vars.csv]

In [2]:
arm_vars = []
with open('./data/arm_vars.csv', 'r', newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        arm_vars.append(row[0])

### Create and Parse Units from arm_vars

In [3]:
arm_vars_units = []
arm_vars_names = []

for idx,var in enumerate(arm_vars):    
    arm_vars_units.append(re.search(r'\[(.*?)\]',arm_vars[idx]).group(1))  

### set a hardcoded variable Names To plot Legibility

In [4]:
arm_vars_long_names = ['Temperature',
 'Relative_Umidity',
 'Wind_Speed',
 'Rain_Intensity_ORG_815_DA',
 'Pressure',
 'Wind_Dir',
 'Rain_Intensity_PWD22',
 'PWV_LOS',
 'LWP_LOS',
 '_',
 'Radiative_Flux',
 'Percent_opaque_TSI',
 'Percent_thin_TSI',
 'Surface_Soil_heat_flux',
 'Wetness',
 'Soil_moisture',
 'Rain_Intensity_Disdrometer',
 'Median_volume_diameter_Disdrometer',
 'PWV_MWR',
 'PWV_MWR_23.835_30.0_GHz',
 'LWP_MWR',
 'LWP_MWR_23.835_30.0_GHz',
 'LCL_MWR',
 'LFC_MWR',
 'CAPE_MWR',
 'CloudBaseHeight_MWR',
 'Upwelling broadband_hemispheric_irradiance_MFRSR',
 'LWP_MWRRET',
 'PWV_MWRRET',
 'Rain_Intensity_Tipping Bucket',
 'Rad_flux_downwelling_shortwave_SKYRAD',
 'Rad_flux_downwelling_longwave_SKYRAD',
 'Rad_flux_upwelling_shortwave_GNDRAD',
 'Rad_flux_upwelling_longwave_GNDRAD',
 'Surface_Energy_Balance_SEBS']

arm_vars_names = ['T',
 'RH',
 'Wind_Speed',
 'RI_ORG_815_DA',
 'P',
 'Wind_Dir',
 'RI_PWD22',
 'PWV_LOS',
 'LWP_LOS',
 '_',
 'Radiative_Flux',
 'pc_opaque_TSI',
 'pc_thin_TSI',
 'Surface_Soil_Heat',
 'Wetness',
 'Soil_moisture',
 'RI_Disdrometer',
 'Median_D_Disdrometer',
 'PWV_MWR',
 'PWV_MWR_23_30_GHz',
 'LWP_MWR',
 'LWP_MWR_23_30_GHz',
 'LCL_MWR',
 'LFC_MWR',
 'CAPE_MWR',
 'CBH_MWR',
 'Upwelling_IRD_MFRSR',
 'LWP_MWRRET',
 'PWV_MWRRET',
 'RI_Tipping_Bucket',
 'Rad_down_shortwave_SKYRAD',
 'Rad_down_longwave_SKYRAD',
 'Rad_up_shortwave_GNDRAD',
 'Rad_up_longwave_GNDRAD',
 'Surface_Energy_Balance']

### map long and variables names with units

In [5]:
dict_var_units      = dict(zip(arm_vars_names, arm_vars_units))
dict_var_short_long = dict(zip(arm_vars_names, arm_vars_long_names))
dict_var_units

{'T': 'degC',
 'RH': '%',
 'Wind_Speed': 'm/s',
 'RI_ORG_815_DA': 'mm/h',
 'P': 'kPa',
 'Wind_Dir': 'º',
 'RI_PWD22': 'mm/h',
 'PWV_LOS': 'mm',
 'LWP_LOS': 'cm',
 '_': '',
 'Radiative_Flux': 'unitless',
 'pc_opaque_TSI': '%',
 'pc_thin_TSI': '%',
 'Surface_Soil_Heat': 'W/m^2',
 'Wetness': 'V',
 'Soil_moisture': '%',
 'RI_Disdrometer': 'mm/h',
 'Median_D_Disdrometer': 'mm',
 'PWV_MWR': 'cm',
 'PWV_MWR_23_30_GHz': 'cm',
 'LWP_MWR': 'mm',
 'LWP_MWR_23_30_GHz': 'mm',
 'LCL_MWR': 'm',
 'LFC_MWR': 'm',
 'CAPE_MWR': 'J/Kg',
 'CBH_MWR': 'm',
 'Upwelling_IRD_MFRSR': 'W/m^2',
 'LWP_MWRRET': 'g/m^2 ',
 'PWV_MWRRET': 'cm',
 'RI_Tipping_Bucket': 'mm/h',
 'Rad_down_shortwave_SKYRAD': 'W/m^2',
 'Rad_down_longwave_SKYRAD': 'W/m^2',
 'Rad_up_shortwave_GNDRAD': 'W/m^2',
 'Rad_up_longwave_GNDRAD': 'W/m^2',
 'Surface_Energy_Balance': 'W/m^2'}

## LOAD Dataframe from 'arm_values.csv' 

In [6]:
df_valores =  pd.read_csv('./data/arm_values.csv',names=arm_vars_names)

df_valores.columns

Index(['T', 'RH', 'Wind_Speed', 'RI_ORG_815_DA', 'P', 'Wind_Dir', 'RI_PWD22',
       'PWV_LOS', 'LWP_LOS', '_', 'Radiative_Flux', 'pc_opaque_TSI',
       'pc_thin_TSI', 'Surface_Soil_Heat', 'Wetness', 'Soil_moisture',
       'RI_Disdrometer', 'Median_D_Disdrometer', 'PWV_MWR',
       'PWV_MWR_23_30_GHz', 'LWP_MWR', 'LWP_MWR_23_30_GHz', 'LCL_MWR',
       'LFC_MWR', 'CAPE_MWR', 'CBH_MWR', 'Upwelling_IRD_MFRSR', 'LWP_MWRRET',
       'PWV_MWRRET', 'RI_Tipping_Bucket', 'Rad_down_shortwave_SKYRAD',
       'Rad_down_longwave_SKYRAD', 'Rad_up_shortwave_GNDRAD',
       'Rad_up_longwave_GNDRAD', 'Surface_Energy_Balance'],
      dtype='object')

## LOAD : [arm_time.csv] 

output : df_valores

Parte Time   :'31-Dec-2013 20:05:06'
Using format : "%d-%b-%Y %H:%M:%S"

In [7]:
arm_time_str = []
with open('./data/arm_time.csv', 'r', newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        arm_time_str.append(row[0])

# PARSE DATE TIME and set INDEX        
df_valores.insert(loc=0, column='time_str', value=arm_time_str)
df_valores.insert(loc=0, column='dt_datetime', value=datetime.datetime.today())
df_valores['dt_datetime'] = datetime.datetime.today()


In [8]:
df_valores['time_str'][1]

'31-Dec-2013 20:05:06'

In [ ]:
for i in np.arange(df_valores.shape[0]):
    df_valores.loc[i,'dt_datetime'] = datetime.datetime.strptime(df_valores['time_str'][i], "%d-%b-%Y %H:%M:%S")


In [ ]:
df_valores.set_index(df_valores["dt_datetime"],inplace=True)
df_valores.head()

In [ ]:
df_valores = df_valores.drop('time_str', axis = 1)

# Preprocessing Stage

Here we will investivestigate about Data:
- Avaiability
- Quality


## Data availability

In [ ]:
nullvalueAbs = df_valores.isnull().sum().values
nullvaluespc = (df_valores.isnull().sum().values/df_valores.shape[0])*100

y_pos = np.arange(len(arm_vars_names))

fig,ax = plt.subplots()
ax.barh(y_pos, nullvaluespc[1:], color = 'blue')
ax.set_yticks(y_pos)
ax.set_yticklabels(arm_vars_names)
ax.set_xlim(0,100)
ax.set_xlabel('%')
plt.suptitle('Missing Data')
plt.gcf().subplots_adjust(left=0.25)
plt.show()

### Drop all TS with null values > 50 ->df_valores_1

OUT: df_valores_1

Its possible to see that a loot of variables dont have any data, so we will work only with TimeSeries that have less then 50% of missing data.  

In [ ]:
df_valores.columns[nullvaluespc > 50]

df_valores_1 = df_valores.copy()
df_valores_1 = df_valores_1.drop(df_valores_1.columns[nullvaluespc > 50],axis=1)
 

In [ ]:
plt.figure(figsize=(14, 14))
for j in np.arange(1,df_valores_1.shape[1]):
    plt.subplot((df_valores_1.shape[1]-1),1,j)
    plt.plot(df_valores_1[df_valores_1.columns[j]].values,label=df_valores_1.columns[j])
    plt.ylabel(" (" + dict_var_units[df_valores_1.columns[j]] + ")")
    plt.legend(loc = 'upper right')
    

###  Data Availability in same time interval

To make this plot we will just set a different number "j" for each valid data, soter in df_window and plot the availability during the field campaing 

In [ ]:
df_window = df_valores_1.copy()

idxs = np.arange(0,df_valores_1.shape[0])

for j in np.arange(1,df_valores_1.shape[1]):
    idxs_notnan = idxs[pd.notnull(df_valores_1.iloc[:,j])]
    df_window.iloc[idxs_notnan,j] = j

In [ ]:
plt.figure(figsize=(8, 8))
plt.yticks(np.arange(2,df_window.shape[1]), df_window.columns[1:])
for j in np.arange(1,df_window.shape[1]):
    plt.plot(df_window[df_window.columns[j]].values)
plt.title('Data Availability')

### SELECT : TimeDate range that all TS has Valid Data --> df_valores_2

OUT: df_valores_2

Its possible to see that exist just a range during the campaing that all TS has data, so we will use a ANDING process in mask_not_null and get the first and the last time that ALL TS have data. After that we will slice df_valore_1 and create df_valores_2 corresponding to that period. 

In [ ]:
mask_not_null = pd.notnull(df_valores_1.iloc[:,1]).values
    
for j in np.arange(2,df_valores_1.shape[1]):
    mask_not_null = mask_not_null & pd.notnull(df_valores_1.iloc[:,j]).values

commom_valid_idx = idxs[mask_not_null]    
first_valid_idx = commom_valid_idx[0]    
last_valid_idx = commom_valid_idx[-1]    

np.arange(0,first_valid_idx)

start = df_valores_1.iloc[first_valid_idx,0]
end =  df_valores_1.iloc[last_valid_idx,0]

df_valores_2 = df_valores_1[(df_valores_1.dt_datetime > start) &  (df_valores_1.dt_datetime < end)].copy()


### Check Data Quality : Visual inspect each TS

In [ ]:
meteo_vars = df_valores_2.columns[np.hstack((np.arange(1,8),10))]
termo_vars = df_valores_2.columns[np.hstack((np.arange(8,10),np.arange(11,19)))]
rad_vars   = df_valores_2.columns[19:]

rain_intensity_vars = meteo_vars[[3,6,7]]
meteo_vars_exclude_rain = set(meteo_vars)-set(rain_intensity_vars)

In [ ]:
plt.figure(figsize=(9, 9))
meteo_vars_exclude_rain = set(meteo_vars)-set(meteo_vars[[3,6,7]])
for idx,var in enumerate(meteo_vars_exclude_rain):
    plt.subplot(len(meteo_vars_exclude_rain),1,idx+1)
    plt.plot(df_valores_2[var].values,label=var)
    plt.ylabel(dict_var_units[var])
    plt.legend(loc = 'upper right')

###  SET NAN to Temperatura < 10, impossible in Central Amazon

In [ ]:
df_valores_2.loc[df_valores_2['T'] < 10] = np.nan

### Correct Offtset of  RH 

In [ ]:
df_valores_2.columns[2]

In [ ]:
for i in np.arange(df_valores_2.shape[0]):
    if df_valores_2.iloc[i,2]  > 100:
        offset = df_valores_2.iloc[i,2] - 100
        df_valores_2.iloc[i,2] = df_valores_2.iloc[i,2] - offset

## RAIN intensity

In [ ]:
plt.figure(figsize=(9, 9))
for idx,var in enumerate(rain_intensity_vars ):
    plt.subplot(len(rain_intensity_vars),1,idx+1)
    plt.plot(df_valores_2[var].values,label=var)
    plt.ylabel(dict_var_units[var])
    plt.legend(loc = 'upper right')
    

### Set NAN to Rain Intensity PWD  > 200

In [ ]:
df_valores_2.loc[df_valores_2['RI_PWD22'] > 200] = np.nan

In [ ]:
plt.figure(figsize=(9, 9))
for idx,var in enumerate(rain_intensity_vars ):
    plt.subplot(len(rain_intensity_vars),1,idx+1)
    plt.plot(df_valores_2[var].values,label=var)
    plt.ylabel(dict_var_units[var])
    plt.legend(loc = 'upper right')


### Drop PWD, too small values  --> df_valores_3

In [ ]:
df_valores_2.columns[7]

In [ ]:
df_valores_3 = df_valores_2.copy()
df_valores_3 = df_valores_3.drop(df_valores_3.columns[7],axis=1)

### Radiometric Variables

In [ ]:
# RAD vars
rad_vars   = df_valores_3.columns[18:]

plt.figure(figsize=(9, 9))
for idx,var in enumerate(rad_vars):
    plt.subplot(len(rad_vars),1,idx+1)
    plt.plot(df_valores_3[var].values,label=var)
    plt.ylabel(dict_var_units[var])
    plt.legend(loc = 'upper right')


## Data Transform 

OUT: df_15min

Due to slow processing to perform Data Exploration the data will be resampled to 15min and will be recorded in df_15min data frame

In [ ]:
df_15min = pd.DataFrame()
for idx,c in enumerate(df_valores_3.columns[1:]):
    df_15min[c] = df_valores_3[c].resample('15min').mean()
df_15min.head(2)

# Data Exploration

In [ ]:
meteo_vars = ['T', 'P', 'RH', 'Wind_Speed','Wind_Dir','CBH_MWR']
rain_vars = ['RI_ORG_815_DA','RI_Disdrometer'] 
termo_vars = ['PWV_LOS', 'LWP_LOS', 'PWV_MWR','PWV_MWR_23_30_GHz', 'LWP_MWR', 'LWP_MWR_23_30_GHz', 'LCL_MWR', 'LFC_MWR', 'CAPE_MWR']
rad_vars = ['Upwelling_IRD_MFRSR','Rad_down_shortwave_SKYRAD','Rad_down_longwave_SKYRAD','Rad_up_shortwave_GNDRAD','Rad_up_longwave_GNDRAD']

## SIMPLE Correlations 

To inspect about linear and instant correlations we used here scatter matrix plot 

In [ ]:
sm = scatter_matrix(df_15min[rad_vars], alpha=0.2,figsize=(12, 10) ,diagonal='kde')
plt.suptitle('Radiometrics Distribution')
#Change label rotation
[s.xaxis.label.set_rotation(45) for s in sm.reshape(-1)]
[s.yaxis.label.set_rotation(0) for s in sm.reshape(-1)]
#May need to offset label when rotating to prevent overlap of figure
[s.get_yaxis().set_label_coords(-0.3,0.5) for s in sm.reshape(-1)]
plt.show()

The scatter plot indicates that in general radiometric variables presents some linear correlation. Its important to inpect by other type of correlations: nonlinear and not instantaneos.
This plot resulted in a fited distribution of each variable, that lays in matrix diagonal. Its possible to see a tunned long wave radiation emittion and a narrow peak of incoming long wave, that indicate the cloud scattering to the ground
Further analysis of shortwave distribution is needed, maybe this distribution indicates a scale-free process

In [ ]:
sm = scatter_matrix(df_15min[meteo_vars], alpha=0.2,figsize=(12, 10), diagonal='kde')
plt.suptitle('Meteo Vars Distribution')
#Change label rotation
[s.xaxis.label.set_rotation(45) for s in sm.reshape(-1)]
[s.yaxis.label.set_rotation(0) for s in sm.reshape(-1)]
#May need to offset label when rotating to prevent overlap of figure
[s.get_yaxis().set_label_coords(-0.3,0.5) for s in sm.reshape(-1)]
plt.show()


In [ ]:
sm = scatter_matrix(df_15min[termo_vars], alpha=0.2, figsize=(12, 10),diagonal='kde')
plt.suptitle('Termo Vars Distribution')
#Change label rotation
[s.xaxis.label.set_rotation(45) for s in sm.reshape(-1)]
[s.yaxis.label.set_rotation(0) for s in sm.reshape(-1)]
#May need to offset label when rotating to prevent overlap of figure
[s.get_yaxis().set_label_coords(-0.3,0.5) for s in sm.reshape(-1)]
plt.show() 

# Load Convective Days

[convective_dates.csv] 

In [ ]:
df_convectives =  pd.read_csv('./data/convective_dates.csv',names=['event_time_str'])

In [ ]:
df_convectives.head()

In [ ]:
as_list = df_convectives.index.tolist()

for i in np.arange(0,len(as_list)):
    tmp = as_list[i]
    as_list[i] = tmp[0:-1]

as_list[1:3]

In [ ]:
df_convectives.insert(loc=0, column='time_str', value=as_list)
df_convectives = df_convectives.drop('event_time_str',axis=1) 

In [ ]:
print(df_convectives.shape[0])
df_convectives.head()

In [ ]:
df_convectives.insert(loc=0, column='dt_datetime', value=datetime.datetime.today())

In [ ]:
for i in np.arange(1,df_convectives.shape[0]):
    df_convectives.iloc[i,0] = datetime.datetime.strptime(df_convectives['time_str'][i], '%d-%b-%Y %H:%M:%S')
df_convectives.set_index(df_convectives["dt_datetime"],inplace=True)    
df_convectives.drop('time_str',axis=1)

start = resampled Datetime of first convective day

end   = resampled Datetime of last convective day

In [ ]:
start = df_15min.index[0]
end = df_15min.index[-1]
start

In [ ]:
df_convectives_1 = df_convectives[ (df_convectives['dt_datetime'] >= start ) & (df_convectives['dt_datetime'] <= end )].copy()
df_convectives_1['dt_datetime'][0]

In [ ]:
df_convectives_1.shape[0]

### Generate Matrix for each TS

| var = T  |D1 |D2 |D3 |... |D326|
|--------------------------------|   
|t1        |   |   |   |    |    |
|t2        |   |   |   |    |    |
|t3        |   |   |   |    |    |
|...       |   |   |   |    |    |
|t23       |   |   |   |    |    |


This kind of structure is used to explore each convective day for each TS

In [ ]:
df_15min.insert(loc=0, column='dt_datetime', value=df_15min.index)

In [ ]:
i = 132 # df_convectives
i = 2   # df_convectives_1
dfSelected = df_15min[ (df_15min['dt_datetime'].dt.day == df_convectives_1.dt_datetime.iloc[i].day) & (df_15min['dt_datetime'].dt.month == df_convectives_1.dt_datetime.iloc[i].month) & (df_15min['dt_datetime'].dt.year == df_convectives_1.dt_datetime.iloc[i].year)].copy()
print(len(dfSelected['T'][:].values))
time_length = len(dfSelected['T'][:].values)
dfSelected['T'][:].values

## Create nMatrix to store TS to each convective day 
Each numpy array has the exact name of df_15min.columns

In [ ]:
din_var = {}
for idx in np.arange(1,len(df_15min.columns)):
    globals()[df_15min.columns[idx]] = np.zeros((time_length,df_convectives_1.shape[0],),dtype=np.float64)
    globals()[str('df_') + df_15min.columns[idx]] = pd.DataFrame()
    

In [ ]:
# TEST
# assing all times of event = 2 in var = 1
print(df_15min.columns[1])
vars()['T'][:,2] = dfSelected['T'][:].values
print(vars()['T'].shape)

In [ ]:
for ievent in np.arange(df_convectives_1.shape[0]):
    dfSelected = df_15min[ (df_15min['dt_datetime'].dt.day == df_convectives_1.dt_datetime.iloc[ievent].day) & (df_15min['dt_datetime'].dt.month == df_convectives_1.dt_datetime.iloc[ievent].month) & (df_15min['dt_datetime'].dt.year == df_convectives_1.dt_datetime.iloc[ievent].year)].copy()
    non_zero_list = []
    for c in np.arange(1,len(df_15min.columns)):
        values_size = dfSelected[dfSelected.columns[c]].values.size
        non_zero = np.count_nonzero(dfSelected[dfSelected.columns[c]].values)
        
        if values_size > 0 :
            #print(dfSelected[dfSelected.columns[c]].values)
            vars()[df_15min.columns[c]][:,ievent] = dfSelected[dfSelected.columns[c]][:].values
            non_zero_list.append(non_zero)
        
    print("Event : " + str(df_convectives.shape[0]-ievent) + "  (" + str(non_zero_list) + ")")

## Cluster

In [ ]:

teste = np.transpose(LFC_MWR)
teste[10,:] == LFC_MWR[:,10]
series = np.asmatrix(teste)

from dtaidistance import dtw

ds = dtw.distance_matrix_fast(series)
print(ds)

### Because of this error we must investigate further to decice what to do with NAN data

In [ ]:
from dtaidistance import clustering

model1 = clustering.Hierarchical(dtw.distance_matrix_fast, {})
model2 = clustering.HierarchicalTree(model1)
model3 = clustering.LinkageTree(dtw.distance_matrix_fast, {})

cluster_idx = model3.fit(series)

## Plot Missing Value by event for each TS and All TS with more than 20 Missing Value per Event

In [ ]:
for idx in np.arange(1,15):

    valores= np.arange(0,vars()[df_15min.columns[idx]].shape[1])
    idx_v = np.arange(0,vars()[df_15min.columns[idx]].shape[1])

    for ievent in np.arange(0,vars()[df_15min.columns[idx]].shape[1]):
        valores[ievent] = np.isnan(vars()[df_15min.columns[idx]][:,ievent]).sum()

    plt.figure()
    plt.plot(valores)
    plt.ylabel(df_15min.columns[idx] + 'Missing Data')
    plt.xlabel('events')
    plt.title(df_15min.columns[idx] + ' Missing Data Count ')

    plt.figure(figsize=(8, 8))
    for i in np.arange(0,len(valores[valores > 10])):
        plt.subplot(len(valores[valores > 10]),1,i+1)
        plt.plot(vars()[df_15min.columns[idx]][:,i],'.')
        plt.suptitle(df_15min.columns[idx] + ' : Events with More than 10 Missing ' )
    

In [ ]:
for idx in np.arange(15,len(df_15min.columns)):

    valores= np.arange(0,vars()[df_15min.columns[idx]].shape[1])
    idx_v = np.arange(0,vars()[df_15min.columns[idx]].shape[1])

    for ievent in np.arange(0,vars()[df_15min.columns[idx]].shape[1]):
        valores[ievent] = np.isnan(vars()[df_15min.columns[idx]][:,ievent]).sum()

    plt.figure()
    plt.plot(valores)
    plt.ylabel(df_15min.columns[idx] + 'Missing Data')
    plt.xlabel('events')
    plt.title(df_15min.columns[idx] + ' Missing Data Count ')

    plt.figure(figsize=(8, 8))
    for i in np.arange(0,len(valores[valores > 10])):
        plt.subplot(len(valores[valores > 10]),1,i+1)
        plt.plot(vars()[df_15min.columns[idx]][:,i],'.')
        plt.suptitle(df_15min.columns[idx] + ' : Events with More than 10 Missing ' )


## Interpolate using akima, repeat last one and fill all other nans with row mean  (same local time)

And finally due to a persistent existence of null values, row mean calculation to substitute in the remaining nan, of each column for the same row.
Remember that each column is a deep convective event and each line is a tn of this day due to periodicity of time series, so each line has the same value of Local Time. The mean valued of each row represents the men value of the variable of an especific Time of day during almost 2 years.


In [ ]:
for idx in np.arange(1,len(df_15min.columns)):
    np_i = vars()[df_15min.columns[idx]]
    vars()[str('df_') + df_15min.columns[idx]] = pd.DataFrame(data=np_i[:],copy=True) 
    df_i = vars()[str('df_') + df_15min.columns[idx]]
    df_i = df_i.interpolate(method='akima',axis=0)
    df_i = df_i.fillna(method='pad', limit=1)
    vars()[str('df_') + df_15min.columns[idx]] = df_i
    m = df_i.mean(axis=1)
    for i, col in enumerate(df_i):
        df_i.iloc[:, i] = df_i.iloc[:, i].fillna(m)    
    print(str(df_i.isna().sum().sum()) + '  :  ' + df_15min.columns[idx])

## Check Results

In [ ]:
for idx in np.arange(1,len(df_15min.columns)):
    vect_i  = vars()[df_15min.columns[idx]]
    df_i    = vars()[str('df_') + df_15min.columns[idx]]
    
    valores = np.arange(0,vect_i.shape[1])
    idx_v   = np.arange(0,vect_i.shape[1])

    for ievent in np.arange(0,vect_i.shape[1]):
        valores[ievent] = np.isnan(vect_i[:,ievent]).sum()

    print(str(df_i.isna().sum().sum()) + '  :  ' + df_15min.columns[idx])
    
    plt.figure(figsize=(8, 8))
    for i in np.arange(0,len(valores[valores > 10])):
        plt.subplot(len(valores[valores > 10]),1,i+1)
        plt.plot(vect_i[:,i],'*-')
        plt.plot(df_i.iloc[:,i].values,'*-')
        plt.suptitle(df_15min.columns[idx] + ' : Events with More than 10 Missing ' )
    plt.show()

## Cluster Again


In [ ]:
for idx in np.arange(1,len(df_15min.columns)):
    vect_i  = vars()[df_15min.columns[idx]]
    df_i    = vars()[str('df_') + df_15min.columns[idx]]
    series  = np.transpose(df_i.as_matrix())
    
    ds = dtw.distance_matrix_fast(series)
    model1 = clustering.Hierarchical(dtw.distance_matrix_fast, {})
    model2 = clustering.HierarchicalTree(model1)
    model3 = clustering.LinkageTree(dtw.distance_matrix_fast, {})
    cluster_idx = model3.fit(series)    
    model2.plot
    model3.plot
    plt.show()

In [ ]:
plt.figure()
model2.plot
plt.show()

In [ ]:
model3.plot

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
for idx in np.arange(1,len(df_15min.columns)):
    df_i    = vars()[str('df_') + df_15min.columns[idx]]
    z = linkage(df_i.iloc[:,0:181],'ward')

    plt.figure(figsize=(10,10))
    dendrogram(z)
    plt.title('Hierarchical Cluster ' + df_15min.columns[idx])
    plt.show()